# Notebook for construction of basic dataframe 
## - uses class data structures (see ia_s2p_preprocess.ipynb)

In [2]:
%run ./rob_setup_notebook.ipynb

import utils.utils_funcs as uf
import utils.gsheets_importer as gsi
import utils.ia_funcs as ia 
from utils.paq2py import *

session_type = 'sensory_fewtargets'
# 'sensory_nodetrend'
# 'sensory_highactivity'
# 'sensory_topactivity'
# 'sensory_topcells'
# 'sensory_extremefilter'
# 'sensory_responsivecells'
# 'sensory_2sec_test'
# 'projection_nodetrend'
# 'projection_2sec_test'
# 'interneuron'
# 'combined'
# 'sensory_fewtargets'

combined = True if 'combined' in session_type else False
sensory = True if 'sensory' in session_type else False
projection = True if 'projection' in session_type else False
interneuron = True if 'interneuron' in session_type else False

if interneuron:
    stim_types = ['pr', 'ps', 'none']
if combined:
    stim_types = ['pr', 'ps', 'none']
else:
    stim_types = ['pr', 'ps', 'none', 'w']

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload

Suite2p path: ['/home/rlees/anaconda3/envs/suite2p/lib/python3.7/site-packages/suite2p']
qnap_path: /home/rlees/mnt/qnap 
qnap_data_path /home/rlees/mnt/qnap/Data 
pkl_folder: /home/rlees/mnt/qnap/pkl_files 
master_path: /home/rlees/mnt/qnap/master_pkl/master_obj.pkl 
fig_save_path: /home/rlees/mnt/qnap/Analysis/Figures 
stam_save_path: /home/rlees/mnt/qnap/Analysis/STA_movies 
s2_borders_path: /home/rlees/mnt/qnap/Analysis/S2_borders


# Define dataframe paths

In [3]:
session_df_path = qnap_path + '/pkl_files/dataframes/' + session_type + '_session_df.pkl'
experiment_df_path = qnap_path + '/pkl_files/dataframes/' + session_type + '_experiment_df.pkl'
trial_df_path = qnap_path + '/pkl_files/dataframes/' + session_type + '_trial_df.pkl'
cell_df_path = qnap_path + '/pkl_files/dataframes/' + session_type + '_cell_df.pkl'
cell_trial_df_path = qnap_path + '/pkl_files/dataframes/' + session_type + '_cell_trial_df.pkl'
cell_mean_timepoint_df_path = qnap_path + '/pkl_files/dataframes/' + session_type + '_cell_mean_timepoint_df.pkl'
cell_trial_single_timepoint_df_path = (qnap_path + '/pkl_files/dataframes/' + session_type + 
                                       '_cell_trial_single_timepoint_df.pkl')

# Data organisation

### For the following dataframes it will be possible to uses the dataframe above to index the dataframe below, or vice versa

## Use cases:
- find average response to pr stim in ps targets
    - use the cell dataframe to get cell IDs for ps targets and index the timepoint dataframe to produce an average response to pr
- find sum or mean dff response on a single trial for pr target cells
    - use the cell dataframe to find pr target cells, use timepoint dataframe to group by trial_num and mean the pre and post

# Construct dataframes

## Session dataframe
- all sessions in project (index = session)
- include information such as:
    - ss_id [string]
    - session_id [string] (sheet_name)
    - s2p_path [string]
    - frame_list [string]
    - x_pix_reg [list] (xoff)
    - y_pix_reg [list] (yoff)
    - mean_img [array]
    - mean_imgE [array]

In [4]:
df_name = session_type + '_session_df'
pkl_files = ia.listdirFullpath(os.path.join(pkl_folder, session_type), string='.pkl')

df = pd.DataFrame()
index = []

for pkl in pkl_files:
        
    ses_obj = ia.loadPickle(pkl)
    
    ss_id = ses_obj.ss_id
    session_id = ses_obj.sheet_name
    s2p_path = ses_obj.s2p_path
    frame_list = list(ses_obj.frame_list)
    
    exp_list = ia.makeExpList(ses_obj, stim_types)

    x_pix_reg = list(exp_list[0].xoff)
    y_pix_reg = list(exp_list[0].yoff)
    mean_img = exp_list[0].mean_img[0]
    mean_imgE = exp_list[0].mean_imgE[0]
            
    temp_df = pd.DataFrame({'ss_id'      : ss_id,
                            's2p_path'   : s2p_path,
                            'frame_list' : [frame_list],
                            'x_pix_reg'  : [x_pix_reg],
                            'y_pix_reg'  : [y_pix_reg],
                            'mean_img'   : [mean_img],
                            'mean_imgE'  : [mean_imgE]
                            })
    
    df = pd.concat([df, temp_df])
    index.extend([session_id])
    
df.index = index
df.index.rename('session_id', inplace=True)

# Pickle the object output to save it for analysis
pkl_path = os.path.join(pkl_folder, 'dataframes', df_name + '.pkl')
df.to_pickle(pkl_path)

clear_output()
df.head()

,ss_id,s2p_path,frame_list,x_pix_reg,y_pix_reg,mean_img,mean_imgE
session_id,,,,,,,
2022-01-12_RL136,1PgSXs6BVHe9dACWEydykucN74rirNfOY4Vr_AmLcCdY,/home/rlees/mnt/qnap/Data/2022-01-12/RL136/202...,"[22836, 22836, 16977, 22986]","[0, 0, 0, 1, 1, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1, ...","[1, 0, 0, 0, 0, 0, 0, 0, -1, -1, -1, -1, -1, -...","[[13.017492847550646, 13.997781281018275, 13.0...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,..."
2022-01-13_RL135,1PgSXs6BVHe9dACWEydykucN74rirNfOY4Vr_AmLcCdY,/home/rlees/mnt/qnap/Data/2022-01-13/RL135/202...,"[22836, 22836, 16826, 22836]","[-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -...","[0, 0, 0, -1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...","[[9.519031101319523, 10.037628612276466, 9.816...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,..."
2022-01-14_RL134,1PgSXs6BVHe9dACWEydykucN74rirNfOY4Vr_AmLcCdY,/home/rlees/mnt/qnap/Data/2022-01-14/RL134/202...,"[22836, 22836, 16826, 22836]","[1, 0, 1, 0, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, ...","[0, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1...","[[10.000867180725152, 11.069831485691516, 10.7...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,..."
2022-01-17_RL133,1PgSXs6BVHe9dACWEydykucN74rirNfOY4Vr_AmLcCdY,/home/rlees/mnt/qnap/Data/2022-01-17/RL133/202...,"[22836, 22836, 16826, 22836]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[[9.283228256029247, 8.790821946703543, 8.9826...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,..."


## Experiment dataframe
- all experiments i.e. stim types across all sessions (index = session, experiment)
- include information such as:
    - session_id (sheet_name)
    - stim_type
    - tiff_path
    - naparm_path
    - paq_path
    - stim_channel
    - fps
    - frame_x
    - frame_y
    - n_planes
    - pix_sz_x
    - pix_sz_y
    - scan_x
    - scan_y
    - zoom
    - n_frames
    - last_good_frame
    - n_groups
    - n_reps
    - n_shots
    - inter_point_delay
    - single_stim_dur
    - spiral_size
    - stim_dur
    - stim_freq
    - duration_frames
    - frames [tuple]
    - n_units
    - num_s1_cells
    - num_s2_cells
    - pre_frames
    - post_frames
    - test_frames
    - pre_trial_frames
    - post_trial_frames
    - n_trials
    - n_groups = exp_obj.n_groups
    - n_reps = exp_obj.n_reps
    - n_shots = exp_obj.n_shots
    - inter_point_delay = exp_obj.inter_point_delay
    - single_stim_dur = exp_obj.single_stim_dur
    - spiral_size = exp_obj.spiral_size
    - n_targets
    - n_targeted_cells
    - sta_euclid_dist

In [5]:
df_name = session_type + '_experiment_df'
pkl_files = ia.listdirFullpath(os.path.join(pkl_folder, session_type), string='.pkl')

df = pd.DataFrame()
index = []

for pkl in pkl_files:
        
    ses_obj = ia.loadPickle(pkl)
    
    exp_list = ia.makeExpList(ses_obj, stim_types)
    
    for exp_obj in exp_list:
        
        session_id = exp_obj.sheet_name
        stim_type = exp_obj.stim_type
        tiff_path = str(exp_obj.tiff_path)
        naparm_path = str(exp_obj.naparm_path)
        paq_path = str(exp_obj.paq_path)
        stim_channel = exp_obj.stim_channel
        fps = exp_obj.fps
        frame_x = exp_obj.frame_x
        frame_y = exp_obj.frame_y
        n_planes = exp_obj.n_planes
        pix_sz_x = exp_obj.pix_sz_x
        pix_sz_y = exp_obj.pix_sz_y
        scan_x = exp_obj.scan_x
        scan_y = exp_obj.scan_y
        zoom = exp_obj.zoom
        n_frames = int(exp_obj.n_frames)
        last_good_frame = int(exp_obj.last_good_frame)
        stim_dur = exp_obj.stim_dur
        stim_freq = exp_obj.stim_freq
        duration_frames = exp_obj.duration_frames
        frames = exp_obj.frames
        n_units = exp_obj.n_units[0]
        num_s1_cells = exp_obj.num_s1_cells[0]
        num_s2_cells = exp_obj.num_s2_cells[0]
        pre_frames = exp_obj.pre_frames
        post_frames = exp_obj.post_frames
        test_frames = exp_obj.test_frames
        pre_trial_frames = exp_obj.pre_trial_frames
        post_trial_frames = exp_obj.post_trial_frames
        n_trials = exp_obj.n_trials
        
        try:
            n_groups = exp_obj.n_groups
            n_reps = exp_obj.n_reps
            n_shots = exp_obj.n_shots
            inter_point_delay = exp_obj.inter_point_delay
            single_stim_dur = exp_obj.single_stim_dur
            spiral_size = exp_obj.spiral_size
            n_targets = exp_obj.n_targets
            n_targeted_cells = int(exp_obj.n_targeted_cells)
            sta_euclid_dist = exp_obj.sta_euclid_dist
        except AttributeError:
            print('no photostim attributes for this experiment', session_id, stim_type)
            n_groups = np.nan
            n_reps = np.nan
            n_shots = np.nan
            inter_point_delay = np.nan
            single_stim_dur = np.nan
            spiral_size = np.nan
            n_targets = np.nan
            n_targeted_cells = np.nan
            sta_euclid_dist = np.nan
        
        temp_df = pd.DataFrame({'session_id' : pd.Series(session_id, dtype='category'),
                                'stim_type'  : pd.Series(stim_type, dtype='category'),
                                'tiff_path' : tiff_path,
                                'naparm_path' : naparm_path,
                                'paq_path' : paq_path,
                                'stim_channel' : stim_channel,
                                'fps' : fps,
                                'frame_x' : frame_x,
                                'frame_y' : frame_y,
                                'n_planes' : n_planes,
                                'pix_sz_x' : pix_sz_x,
                                'pix_sz_y' : pix_sz_y,
                                'scan_x' : scan_x,
                                'scan_y' : scan_y,
                                'zoom' : zoom,
                                'n_frames' : n_frames,
                                'last_good_frame' : last_good_frame,
                                'stim_dur' : stim_dur,
                                'stim_freq' : stim_freq,
                                'duration_frames' : duration_frames,
                                'frames' : [frames],
                                'n_units' : n_units,
                                'num_s1_cells' : num_s1_cells,
                                'num_s2_cells' : num_s2_cells,
                                'pre_frames' : pre_frames,
                                'post_frames' : post_frames,
                                'test_frames' : test_frames,
                                'pre_trial_frames' : pre_trial_frames,
                                'post_trial_frames' : post_trial_frames,
                                'n_trials' : n_trials,
                                'sta_euclid_dist' : sta_euclid_dist,
                                'n_groups' : n_groups,
                                'n_reps' : n_reps,
                                'n_shots' : n_shots,
                                'inter_point_delay' : inter_point_delay,
                                'single_stim_dur' : single_stim_dur,
                                'spiral_size' : spiral_size,
                                'n_targets' : n_targets,
                                'n_targeted_cells' : n_targeted_cells
                                })
        
        df = pd.concat([df, temp_df])
        experiment_id = session_id + '_' + stim_type
        index.extend([experiment_id])

df.index = index
df.index.rename('experiment_id', inplace=True)

# Pickle the object output to save it for analysis
pkl_path = os.path.join(pkl_folder, 'dataframes', df_name + '.pkl')
df.to_pickle(pkl_path)

clear_output()
df.head()

,session_id,stim_type,tiff_path,naparm_path,paq_path,stim_channel,fps,frame_x,frame_y,n_planes,...,n_trials,sta_euclid_dist,n_groups,n_reps,n_shots,inter_point_delay,single_stim_dur,spiral_size,n_targets,n_targeted_cells
experiment_id,,,,,,,,,,,,,,,,,,,,,
2022-01-12_RL136_pr,2022-01-12_RL136,pr,/home/rlees/mnt/qnap/Data/2022-01-12/RL136/202...,/home/rlees/mnt/qnap/Data/2022-01-12/RL136/202...,/home/rlees/mnt/qnap/Data/2022-01-12/RL136/202...,markpoints2packio,15.023206,1024,1024,1,...,100,104.900392,1.0,10.0,1.0,5.0,10.0,10.0,16.0,41.0
2022-01-12_RL136_ps,2022-01-12_RL136,ps,/home/rlees/mnt/qnap/Data/2022-01-12/RL136/202...,/home/rlees/mnt/qnap/Data/2022-01-12/RL136/202...,/home/rlees/mnt/qnap/Data/2022-01-12/RL136/202...,markpoints2packio,15.023207,1024,1024,1,...,100,89.725054,1.0,10.0,1.0,5.0,10.0,10.0,16.0,50.0
2022-01-12_RL136_none,2022-01-12_RL136,none,/home/rlees/mnt/qnap/Data/2022-01-12/RL136/202...,/home/rlees/mnt/qnap/Data/2022-01-12/RL136/202...,/home/rlees/mnt/qnap/Data/2022-01-12/RL136/202...,markpoints2packio,15.023206,1024,1024,1,...,100,NaN,1.0,10.0,1.0,5.0,10.0,10.0,16.0,41.0
2022-01-12_RL136_w,2022-01-12_RL136,w,/home/rlees/mnt/qnap/Data/2022-01-12/RL136/202...,none,/home/rlees/mnt/qnap/Data/2022-01-12/RL136/202...,piezo_stim,15.023206,1024,1024,1,...,100,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2022-01-13_RL135_pr,2022-01-13_RL135,pr,/home/rlees/mnt/qnap/Data/2022-01-13/RL135/202...,/home/rlees/mnt/qnap/Data/2022-01-13/RL135/202...,/home/rlees/mnt/qnap/Data/2022-01-13/RL135/202...,markpoints2packio,15.023206,1024,1024,1,...,100,93.538023,1.0,10.0,1.0,5.0,10.0,10.0,16.0,40.0


## Trial dataframe
- all trials across all experiments i.e. stim types for all sessions (index = session, experiment, trial)
    - session_id (sheet_name)
    - stim_type
    - trial_num [float]
    - trial_target_dff
    - trial_target_euclid_dist

In [6]:
df_name = session_type + '_trial_df'
pkl_files = ia.listdirFullpath(os.path.join(pkl_folder, session_type), string='.pkl')

df = pd.DataFrame()
index = []

for pkl in pkl_files:
        
    ses_obj = ia.loadPickle(pkl)
    
    exp_list = ia.makeExpList(ses_obj, stim_types)
    
    for exp_obj in exp_list:
        
        session_id = list(np.repeat(exp_obj.sheet_name, exp_obj.n_trials))
        stim_type = list(np.repeat(exp_obj.stim_type, exp_obj.n_trials))
        experiment_id = [session + '_' + stim for session, stim in zip(session_id,stim_type)]

        trial_num = list(range(1,exp_obj.n_trials+1))
        trial_num_str = np.array(trial_num, dtype='str')
        trial_id = [exp + '_' + trial for exp, trial in zip(experiment_id, trial_num_str)]
        
        try:
            trial_target_dff = exp_obj.trial_target_dff
            trial_target_euclid_dist = exp_obj.trial_euclid_dist
        except AttributeError:
            print('no photostim attributes for this experiment', session_id[0], stim_type[0])
            trial_target_dff = list(np.repeat(np.nan, exp_obj.n_trials))
            trial_target_euclid_dist = list(np.repeat(np.nan, exp_obj.n_trials))
        
        temp_df = pd.DataFrame({'session_id' : pd.Series(session_id, dtype='category'),
                                'stim_type'  : pd.Series(stim_type, dtype='category'),
                                'experiment_id' : experiment_id,
                                'trial_num'  : pd.Series(trial_num, dtype='int8'),
                                'trial_target_dff' : trial_target_dff,
                                'trial_target_euclid_dist' : trial_target_euclid_dist
                                })
        
        df = pd.concat([df, temp_df])
        index.extend(trial_id)
    
df.index = index
df.index.rename('trial_id', inplace=True)
    
# Pickle the object output to save it for analysis
pkl_path = os.path.join(pkl_folder, 'dataframes', df_name + '.pkl')
df.to_pickle(pkl_path)

clear_output()
df.head()

,session_id,stim_type,experiment_id,trial_num,trial_target_dff,trial_target_euclid_dist
trial_id,,,,,,
2022-01-12_RL136_pr_1,2022-01-12_RL136,pr,2022-01-12_RL136_pr,1,1.473476,98.208215
2022-01-12_RL136_pr_2,2022-01-12_RL136,pr,2022-01-12_RL136_pr,2,5.696982,110.846493
2022-01-12_RL136_pr_3,2022-01-12_RL136,pr,2022-01-12_RL136_pr,3,4.700877,87.827107
2022-01-12_RL136_pr_4,2022-01-12_RL136,pr,2022-01-12_RL136_pr,4,4.899731,96.099906
2022-01-12_RL136_pr_5,2022-01-12_RL136,pr,2022-01-12_RL136_pr,5,5.882133,80.932828


## Cell dataframe
- all cells across all sessions (index = session, cell)
- include information such as:
    - session_id (sheet_name)
    - cell_num
    - cell_id [float]
    - s1_cell [bool]
    - s2_cell [bool]
    - pr_target [bool]
    - ps_target [bool]
    - pr_resp_sign [string]
    - ps_resp_sign [string]
    - spont_resp_sign [string]
    - whisker_resp_sign [string]
    - cell_med [tuple]
    - cell_x [list]
    - cell_y [list]
    - prob_response_pr [float]
    - prob_response_ps [float]
    - prob_response_whisker [float]
    - prob_response_spont
    - sta_sig_pr [bool]
    - sta_sig_ps [bool]
    - sta_sig_whisker [bool]
    - sta_sig_spont [bool]
    - sta_sig_nomulti_pr [bool]
    - sta_sig_nomulti_ps [bool]
    - sta_sig_nomulti_whisker [bool]
    - sta_sig_nomulti [bool]
    - wilcoxons

In [7]:
df_name = session_type + '_cell_df'
pkl_files = ia.listdirFullpath(os.path.join(pkl_folder, session_type), string='.pkl')

df = pd.DataFrame()
index = []

for pkl in pkl_files:
        
    ses_obj = ia.loadPickle(pkl)
    
    exp_list = ia.makeExpList(ses_obj, stim_types)
    
    session_id = list(np.repeat(exp_list[0].sheet_name, exp_list[0].n_units[0]))
    
    cell_num = exp_list[0].cell_id[0]
    cell_num_str = np.array(cell_num, dtype='str')
    cell_id = [session + '_' + cell for session, cell in zip(session_id, cell_num_str)]
    
    s1_cell = exp_list[0].cell_s1[0]
    s2_cell = exp_list[0].cell_s2[0]
    cell_med = exp_list[0].cell_med[0]
    cell_x = exp_list[0].cell_x[0]
    cell_y = exp_list[0].cell_y[0]
    
    for exp_obj in exp_list:
        
        if exp_obj.stim_type == 'pr':
            pr_target = exp_obj.targeted_cells
            pr_resp_pos = exp_obj.sta_amplitudes[0] > 0
            pr_sta_amp_resp = exp_obj.sta_amplitudes[0]
            prob_response_pr = exp_obj.prob_response[0]
            sta_sig_pr = exp_obj.sta_sig[0]
            sta_sig_nomulti_pr = exp_obj.sta_sig_nomulti[0]
            wilcoxons_pr = exp_obj.wilcoxons[0]            
        
        if exp_obj.stim_type == 'ps':
            ps_target = exp_obj.targeted_cells
            ps_resp_pos = exp_obj.sta_amplitudes[0] > 0
            ps_sta_amp_resp = exp_obj.sta_amplitudes[0]
            prob_response_ps = exp_obj.prob_response[0]
            sta_sig_ps = exp_obj.sta_sig[0]
            sta_sig_nomulti_ps = exp_obj.sta_sig_nomulti[0]
            wilcoxons_ps = exp_obj.wilcoxons[0]
        
        if exp_obj.stim_type == 'none':
            spont_resp_pos = exp_obj.sta_amplitudes[0] > 0
            spont_sta_amp_resp = exp_obj.sta_amplitudes[0]
            prob_response_spont = exp_obj.prob_response[0]
            sta_sig_spont = exp_obj.sta_sig[0]
            sta_sig_nomulti_spont = exp_obj.sta_sig_nomulti[0]
            wilcoxons_spont = exp_obj.wilcoxons[0]
        
        if len(exp_list) == 4:
            if exp_obj.stim_type == 'w':
                if not sensory:
                    whisker_sta_amp_resp = exp_obj.sta_amplitudes[0]
                else:
                    whisker_resp_pos = exp_obj.sta_amplitudes[0] > 0
                    whisker_sta_amp_resp = exp_obj.sta_amplitudes[0]
                    prob_response_whisker = exp_obj.prob_response[0]
                    sta_sig_whisker = exp_obj.sta_sig[0]
                    sta_sig_nomulti_whisker = exp_obj.sta_sig_nomulti[0]
                    wilcoxons_whisker = exp_obj.wilcoxons[0]
        else:
            whisker_sta_amp_resp = np.repeat(np.nan, len(exp_obj.sta_amplitudes[0]))
        
    temp_df = pd.DataFrame({'session_id' : pd.Series(session_id, dtype='category'),
                            'cell_num' : pd.Series(cell_num, dtype='int16'),
                            's1_cell' : s1_cell,
                            's2_cell' : s2_cell,
                            'cell_med' : cell_med,
                            'cell_x' : cell_x,
                            'cell_y' : cell_y,
                            'pr_target' : pr_target,
                            'ps_target' : ps_target,
                            'pr_sta_amp_resp' : pr_sta_amp_resp,
                            'ps_sta_amp_resp' : ps_sta_amp_resp,
                            'spont_sta_amp_resp' : spont_sta_amp_resp,
                            'whisker_sta_amp_resp' : whisker_sta_amp_resp,
                            'pr_resp_sign' : pr_resp_pos,
                            'ps_resp_sign' : ps_resp_pos,
                            'spont_resp_sign' : spont_resp_pos,
                            'prob_response_pr' : prob_response_pr,
                            'prob_response_ps' : prob_response_ps,
                            'prob_response_spont' : prob_response_spont,
                            'sta_sig_pr' : sta_sig_pr,
                            'sta_sig_ps' : sta_sig_ps,
                            'sta_sig_spont' : sta_sig_spont,
                            'sta_sig_nomulti_pr' : sta_sig_nomulti_pr,
                            'sta_sig_nomulti_ps' : sta_sig_nomulti_ps,
                            'sta_sig_nomulti_spont' : sta_sig_nomulti_spont,
                            'wilcoxons_pr' : wilcoxons_pr,
                            'wilcoxons_ps' : wilcoxons_ps,
                            'wilcoxons_spont' : wilcoxons_spont
                            })
    
    if sensory:
        temp_df2 = pd.DataFrame({'whisker_resp_sign' : whisker_resp_pos,
                                 'prob_response_whisker' : prob_response_whisker,
                                 'sta_sig_whisker' : sta_sig_whisker,
                                 'sta_sig_nomulti_whisker' : sta_sig_nomulti_whisker,
                                 'wilcoxons_whisker' : wilcoxons_whisker
                                })
    else:
        temp_df2 = pd.DataFrame()
    
    new_df = pd.concat([temp_df, temp_df2], axis=1)
    df = pd.concat([df, new_df])
    index.extend(cell_id)
    
df.index = index
df.index.rename('cell_id', inplace=True)

# Pickle the object output to save it for analysis
pkl_path = os.path.join(pkl_folder, 'dataframes', df_name + '.pkl')
df.to_pickle(pkl_path)

clear_output()
df.head()

,session_id,cell_num,s1_cell,s2_cell,cell_med,cell_x,cell_y,pr_target,ps_target,pr_sta_amp_resp,...,sta_sig_nomulti_ps,sta_sig_nomulti_spont,wilcoxons_pr,wilcoxons_ps,wilcoxons_spont,whisker_resp_sign,prob_response_whisker,sta_sig_whisker,sta_sig_nomulti_whisker,wilcoxons_whisker
cell_id,,,,,,,,,,,,,,,,,,,,,
2022-01-12_RL136_0,2022-01-12_RL136,0,False,True,"[821.0, 297.0]","[295, 296, 297, 292, 293, 294, 295, 296, 297, ...","[815, 815, 815, 816, 816, 816, 816, 816, 816, ...",False,False,-0.086202,...,False,False,0.005583,0.063846,0.589323,False,0.39,True,True,0.000102
2022-01-12_RL136_1,2022-01-12_RL136,1,False,True,"[741.0, 480.0]","[476, 477, 478, 479, 480, 474, 475, 476, 477, ...","[736, 736, 736, 736, 736, 737, 737, 737, 737, ...",False,False,-0.090265,...,False,True,0.105350,0.705270,0.031912,False,0.31,True,True,0.001635
2022-01-12_RL136_2,2022-01-12_RL136,2,True,False,"[284.0, 461.0]","[458, 459, 460, 461, 462, 463, 464, 465, 457, ...","[280, 280, 280, 280, 280, 280, 280, 280, 281, ...",False,False,0.052122,...,False,False,0.598844,0.647457,0.770090,True,0.49,False,False,0.480899
2022-01-12_RL136_3,2022-01-12_RL136,3,False,True,"[886.0, 701.0]","[699, 700, 697, 698, 699, 700, 701, 702, 703, ...","[881, 881, 882, 882, 882, 882, 882, 882, 882, ...",False,False,0.107789,...,False,False,0.286478,0.874328,0.356805,False,0.25,False,True,0.014090
2022-01-12_RL136_4,2022-01-12_RL136,4,True,False,"[39.0, 154.0]","[150, 151, 152, 153, 154, 155, 156, 150, 151, ...","[35, 35, 35, 35, 35, 35, 35, 36, 36, 36, 36, 3...",False,False,0.039930,...,True,True,0.157611,0.007550,0.027527,True,0.70,True,True,0.028014


## Cell trial dataframe
- all trials across all cells for all sessions (index = session, cell, trial)
- include information such as:
    - session_id (sheet_name)
    - cell_num
    - cell_id
    - trial_num [float]
    - trial_sig_dff [bool]
    - trial_sig_dfsf [bool]
    - amp_resp

In [8]:
df_name = session_type + '_cell_trial_df'
pkl_files = ia.listdirFullpath(os.path.join(pkl_folder, session_type), string='.pkl')

df = pd.DataFrame()
index = []

for pkl in pkl_files:
        
    ses_obj = ia.loadPickle(pkl)
    
    exp_list = ia.makeExpList(ses_obj, stim_types)
    
    session_id = list(np.repeat(exp_list[0].sheet_name, exp_list[0].n_units[0] * 100))

    cell_num = np.repeat(exp_list[0].cell_id[0], 100, axis=0)
    cell_num_str = np.array(cell_num, dtype='str')
    cell_id = [session + '_' + cell for session, cell in zip(session_id, cell_num_str)]

    trial_num = np.tile(np.arange(1,101), len(exp_list[0].cell_id[0]))
    trial_num_str = np.array(trial_num, dtype='str')

    cell_trial_id = [session + '_' + cell + '_' + trial for session, cell, trial in zip(session_id, cell_num_str, trial_num_str)]
    
    for exp_obj in exp_list:
        
        if exp_obj.stim_type == 'pr':
            trial_sig_dff_pr = pd.Series(exp_obj.trial_sig_dff[0].flatten()).reindex(np.arange(0,len(trial_num)))
            trial_sig_dfsf_pr = pd.Series(exp_obj.trial_sig_dfsf[0].flatten()).reindex(np.arange(0,len(trial_num)))
            pr_amp_resp = pd.Series(exp_obj.all_amplitudes[0].flatten()).reindex(np.arange(0,len(trial_num)))
            
        if exp_obj.stim_type == 'ps':
            trial_sig_dff_ps = pd.Series(exp_obj.trial_sig_dff[0].flatten()).reindex(np.arange(0,len(trial_num)))
            trial_sig_dfsf_ps = pd.Series(exp_obj.trial_sig_dfsf[0].flatten()).reindex(np.arange(0,len(trial_num)))
            ps_amp_resp = pd.Series(exp_obj.all_amplitudes[0].flatten()).reindex(np.arange(0,len(trial_num)))
                    
        if exp_obj.stim_type == 'none':
            trial_sig_dff_spont = pd.Series(exp_obj.trial_sig_dff[0].flatten()).reindex(np.arange(0,len(trial_num)))
            trial_sig_dfsf_spont = pd.Series(exp_obj.trial_sig_dfsf[0].flatten()).reindex(np.arange(0,len(trial_num)))
            spont_amp_resp = pd.Series(exp_obj.all_amplitudes[0].flatten()).reindex(np.arange(0,len(trial_num)))
        
        if sensory:
            if exp_obj.stim_type == 'w':
                trial_sig_dff_whisker = pd.Series(exp_obj.trial_sig_dff[0].flatten()).reindex(np.arange(0,len(trial_num)))
                trial_sig_dfsf_whisker = pd.Series(exp_obj.trial_sig_dfsf[0].flatten()).reindex(np.arange(0,len(trial_num)))
                whisker_amp_resp = pd.Series(exp_obj.all_amplitudes[0].flatten()).reindex(np.arange(0,len(trial_num)))
                    
    temp_df = pd.DataFrame({'session_id' : pd.Series(session_id, dtype='category'),
                            'cell_num' : pd.Series(cell_num, dtype='int16'),
                            'cell_id' : pd.Series(cell_id, dtype='category'),
                            'trial_num' : pd.Series(trial_num, dtype='int8'),
                            'pr_amp_resp' : pr_amp_resp,
                            'ps_amp_resp' : ps_amp_resp,
                            'spont_amp_resp' : spont_amp_resp,
                            'trial_sig_dff_pr' : trial_sig_dff_pr,
                            'trial_sig_dfsf_pr' : trial_sig_dfsf_pr,
                            'trial_sig_dff_ps' : trial_sig_dff_ps,
                            'trial_sig_dfsf_ps' : trial_sig_dfsf_ps,
                            'trial_sig_dff_spont' : trial_sig_dff_spont,
                            'trial_sig_dfsf_spont' : trial_sig_dfsf_spont,
                            })
    
    if sensory:
        temp_df2 = pd.DataFrame({'whisker_amp_resp' : whisker_amp_resp,
                                 'trial_sig_dff_whisker' : trial_sig_dff_whisker,
                                 'trial_sig_dfsf_whisker' : trial_sig_dfsf_whisker
                                })
    else:
        temp_df2 = pd.DataFrame()

    new_df = pd.concat([temp_df, temp_df2], axis=1)
    df = pd.concat([df, new_df])
    index.extend(cell_id)
    
df.index = index
df.index.rename('cell_trial_id', inplace=True)

# Pickle the object output to save it for analysis
pkl_path = os.path.join(pkl_folder, 'dataframes', df_name + '.pkl')
df.to_pickle(pkl_path)

clear_output()
df.head()

,session_id,cell_num,cell_id,trial_num,pr_amp_resp,ps_amp_resp,spont_amp_resp,trial_sig_dff_pr,trial_sig_dfsf_pr,trial_sig_dff_ps,trial_sig_dfsf_ps,trial_sig_dff_spont,trial_sig_dfsf_spont,whisker_amp_resp,trial_sig_dff_whisker,trial_sig_dfsf_whisker
cell_trial_id,,,,,,,,,,,,,,,,
2022-01-12_RL136_0,2022-01-12_RL136,0,2022-01-12_RL136_0,1,-0.010316,-0.111998,-0.446567,False,False,False,False,True,False,0.002748,False,False
2022-01-12_RL136_0,2022-01-12_RL136,0,2022-01-12_RL136_0,2,-0.784273,-0.201098,-0.928362,False,False,False,False,False,False,-0.580391,True,False
2022-01-12_RL136_0,2022-01-12_RL136,0,2022-01-12_RL136_0,3,0.024326,-0.009081,0.033032,False,False,False,False,False,False,-0.854460,False,False
2022-01-12_RL136_0,2022-01-12_RL136,0,2022-01-12_RL136_0,4,0.002661,-0.416168,-0.022260,False,False,True,False,False,False,-0.054463,False,False
2022-01-12_RL136_0,2022-01-12_RL136,0,2022-01-12_RL136_0,5,0.015965,-0.308986,-0.127810,False,False,True,False,False,False,0.029814,False,False


## Cell mean timepoint dataframe
- all timepoints across all cells for all trial averages i.e. sessions (index = session, cell, timepoint)
- include information such as:
    - session_id (sheet_name)
    - cell_num
    - cell_id [float]
    - timepoint [timepoints] (a.k.a. time)
    - pr_sta_resp [dff]
    - ps_sta_resp [dff]
    - w_sta_resp [dff]
    - spont_sta_resp [dff]

In [9]:
df_name = session_type + '_cell_mean_timepoint_df'
pkl_files = ia.listdirFullpath(os.path.join(pkl_folder, session_type), string='.pkl')

df = pd.DataFrame()

for pkl in pkl_files:
        
    ses_obj = ia.loadPickle(pkl)
    
    exp_list = ia.makeExpList(ses_obj, stim_types)
    
    session_id = list(np.repeat(exp_list[0].sheet_name, exp_list[0].n_units[0] * len(exp_list[0].time)))
    
    cell_num = np.repeat(exp_list[0].cell_id[0], len(exp_list[0].time), axis=0)
    cell_id = np.array(cell_num, dtype='str')
    cell_id = np.char.add(np.char.add(session_id, np.repeat('_', len(session_id))), cell_id)
    
    timepoint = np.tile(exp_list[0].time, exp_list[0].n_units[0])
    frame = np.tile(np.arange(len(exp_list[0].time)), exp_list[0].n_units[0])
    
    for exp_obj in exp_list:
        
        if exp_obj.stim_type == 'pr':
            pr_sta_resp = exp_obj.stas[0].flatten()
            
        if exp_obj.stim_type == 'ps':
            ps_sta_resp = exp_obj.stas[0].flatten()
                    
        if exp_obj.stim_type == 'none':
            spont_sta_resp = exp_obj.stas[0].flatten()
                    
        if sensory:
            if exp_obj.stim_type == 'w':
                whisker_sta_resp = exp_obj.stas[0].flatten()

    temp_df = pd.DataFrame({'session_id' : pd.Series(session_id, dtype='category'),
                            'cell_num' : pd.Series(cell_num, dtype='int16'),
                            'cell_id' : pd.Series(cell_id, dtype='category'),
                            'frame' : pd.Series(frame, dtype='int16'),
                            'timepoint' : pd.Series(timepoint, dtype='float32').round(2),
                            'pr_sta_resp' : pr_sta_resp,
                            'ps_sta_resp' : ps_sta_resp,
                            'spont_sta_resp' : spont_sta_resp
                            })

    if sensory:
        temp_df2 = pd.DataFrame({'whisker_sta_resp' : whisker_sta_resp})
    else:
        temp_df2 = pd.DataFrame()
        
    new_df = pd.concat([temp_df, temp_df2], axis=1)
    df = pd.concat([df, new_df])

df.reset_index(drop=True)

# Pickle the object output to save it for analysis
pkl_path = os.path.join(pkl_folder, 'dataframes', df_name + '.pkl')
df.to_pickle(pkl_path)

clear_output()
df.head()

,session_id,cell_num,cell_id,frame,timepoint,pr_sta_resp,ps_sta_resp,spont_sta_resp,whisker_sta_resp
0,2022-01-12_RL136,0,2022-01-12_RL136_0,0,-2.06,-0.015211,0.025294,0.105037,-0.023132
1,2022-01-12_RL136,0,2022-01-12_RL136_0,1,-2.00,-0.014509,0.032983,0.077111,0.016835
2,2022-01-12_RL136,0,2022-01-12_RL136_0,2,-1.93,-0.007853,0.057481,0.055025,-0.011440
3,2022-01-12_RL136,0,2022-01-12_RL136_0,3,-1.86,-0.013354,0.083277,0.057017,0.024051
4,2022-01-12_RL136,0,2022-01-12_RL136_0,4,-1.80,-0.060243,0.074562,0.048369,0.042342


## Cell trial single timepoint dataframe
- all timepoints across all cells for all trials (index = session, cell, trial, timepoint)
- include information such as:
    - session_id (sheet_name)
    - cell_num
    - cell_id [float]
    - trial_num [float]
    - timepoint [timepoints] (a.k.a. time)
    - pr_resp [dff]
    - ps_resp [dff]
    - w_resp [dff]
    - spont_resp [dff]
- include later?:
    - pr_amp_resp
    - ps_amp_resp
    - w_amp_resp
    - spont_amp_resp

In [10]:
df_name = session_type + '_cell_trial_single_timepoint_df'
pkl_files = ia.listdirFullpath(os.path.join(pkl_folder, session_type), string='.pkl')

df = pd.DataFrame()

for pkl in pkl_files:
        
    ses_obj = ia.loadPickle(pkl)
    
    exp_list = ia.makeExpList(ses_obj, stim_types)
    
    session_id = list(np.repeat(exp_list[0].sheet_name, exp_list[0].n_units[0] * 100 * len(exp_list[0].time)))
    
    cell_num = np.repeat(exp_list[0].cell_id[0], 100 * len(exp_list[0].time), axis=0)
    cell_id = np.array(cell_num, dtype='str')
    cell_id = np.char.add(np.char.add(session_id, np.repeat('_', len(session_id))), cell_id)
    
    trial_num = np.tile(np.repeat(np.arange(1,101), len(exp_list[0].time)), len(exp_list[0].cell_id[0]))
    
    timepoint = np.tile(exp_list[0].time, exp_list[0].n_units[0] * 100)
    frame = np.tile(np.arange(len(exp_list[0].time)), exp_list[0].n_units[0] * 100)
    
    for exp_obj in exp_list:
        
        all_trials = np.swapaxes(exp_obj.all_trials[0], 1, 2)
        
        if exp_obj.stim_type == 'pr':
            pr_resp = pd.Series(all_trials.flatten()).reindex(np.arange(0,len(trial_num)))
            
        if exp_obj.stim_type == 'ps':
            ps_resp = pd.Series(all_trials.flatten()).reindex(np.arange(0,len(trial_num)))
                    
        if exp_obj.stim_type == 'none':
            spont_resp = pd.Series(all_trials.flatten()).reindex(np.arange(0,len(trial_num)))
        
        if sensory:
            if exp_obj.stim_type == 'w':
                whisker_resp = pd.Series(all_trials.flatten()).reindex(np.arange(0,len(trial_num)))

    temp_df = pd.DataFrame({'session_id' : pd.Series(session_id, dtype='category'),
                            'cell_num' : pd.Series(cell_num, dtype='int16'),
                            'cell_id' : pd.Series(cell_id, dtype='category'),
                            'trial_num' : pd.Series(trial_num, dtype='int8'),
                            'timepoint' : pd.Series(timepoint, dtype='float32').round(2),
                            'pr_resp' : pr_resp,
                            'ps_resp' : ps_resp,
                            'spont_resp' : spont_resp
                            })
    
    if sensory:
        temp_df2 = pd.DataFrame({'whisker_resp' : whisker_resp})
    else:
        temp_df2 = pd.DataFrame()
        
    new_df = pd.concat([temp_df, temp_df2], axis=1)
    df = pd.concat([df, new_df])
    
df.reset_index(drop=True)

# Pickle the object output to save it for analysis
pkl_path = os.path.join(pkl_folder, 'dataframes', df_name + '.pkl')
df.to_pickle(pkl_path)

clear_output()
df.head()

,session_id,cell_num,cell_id,trial_num,timepoint,pr_resp,ps_resp,spont_resp,whisker_resp
0,2022-01-12_RL136,0,2022-01-12_RL136_0,1,-2.06,-0.037825,0.039950,0.698969,-0.152618
1,2022-01-12_RL136,0,2022-01-12_RL136_0,1,-2.00,-0.079897,0.063336,0.115628,-0.009062
2,2022-01-12_RL136,0,2022-01-12_RL136_0,1,-1.93,0.002241,-0.007508,0.679718,-0.243236
3,2022-01-12_RL136,0,2022-01-12_RL136_0,1,-1.86,0.027903,0.095253,0.038438,0.304176
4,2022-01-12_RL136,0,2022-01-12_RL136_0,1,-1.80,-0.246341,0.130902,0.441270,0.046263
